In [21]:
import os
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

In [22]:
# Inicialize o modelo FaceNet
model = load_model('facenet_keras.h5')

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
print("Modelo carregado com sucesso!")

Modelo carregado com sucesso!


In [23]:
def process_image(image_path, size=(160, 160)):
    """
    Carrega e redimensiona uma imagem para o tamanho esperado pelo modelo FaceNet.
    """
    try:
        image = Image.open(image_path).convert("RGB")
        image = image.resize(size)  # Redimensiona para 160x160
        image_array = np.asarray(image)
        image_array = (image_array - 127.5) / 128.0  # Normalização como esperado pelo modelo FaceNet
        return image_array
    except Exception as e:
        print(f"Erro ao processar a imagem {image_path}: {e}")
        return None

In [24]:
def generate_embeddings_from_directory(directory, model):
    """
    Processa todas as imagens em um diretório e gera embeddings.
    """
    embeddings = []
    labels = []
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if not os.path.isdir(subdir_path):
            continue
        
        print(f"Processando diretório: {subdir}")
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            
            # Processa a imagem
            image = process_image(file_path)
            if image is not None:
                # Gera o embedding usando predict
                image = np.expand_dims(image, axis=0)  # Adiciona uma dimensão para batch
                embedding = model.predict(image)
                embeddings.append(embedding[0])  # Pega apenas o vetor de 128 dimensões
                labels.append(subdir)
    
    return np.array(embeddings), np.array(labels)

In [25]:
# Diretório principal
validation_directory = "C:\\datasets\\fotos\\validation"

In [26]:
# Gerar embeddings
embeddings, labels = generate_embeddings_from_directory(validation_directory, model)

Processando diretório: fotosA
Processando diretório: fotosW


In [27]:
# Salvar os embeddings e os rótulos em um arquivo CSV
import pandas as pd

df = pd.DataFrame(embeddings)
df['label'] = labels
df.to_csv('validation_embeddings.csv', index=False)

print("Embeddings gerados e salvos no arquivo 'validation_embeddings.csv'.")

Embeddings gerados e salvos no arquivo 'validation_embeddings.csv'.
